In [1]:
import pandas as pd
import numpy as np
from Bio.SeqUtils.ProtParam import ProteinAnalysis
from Bio import SeqIO
from collections import Counter
import math
import re

In [2]:
def extract_basic_features(seq):
    """
    Calcola feature direttamente dalla sequenza senza usare dizionari/scale.
    Restituisce un dizionario con feature e descrizione.
    """
    length = len(seq)  # lunghezza totale della sequenza
    counts = Counter(seq)
    
    # Percentuale di ogni amminoacido
    aa_percent = {f"%{aa}": counts[aa]/length for aa in counts}
    
    # Numero di residui acidi
    num_acidi = sum(1 for aa in seq if aa in "DE")
    
    # Numero di residui basici
    num_basici = sum(1 for aa in seq if aa in "KRH")
    
    # Numero di residui polari
    num_polari = sum(1 for aa in seq if aa in "NQSTY")
    
    # Numero di residui idrofobici
    num_idrofobi = sum(1 for aa in seq if aa in "AILMFWV")
    
    # Lunghezza del blocco più lungo di residui ripetuti consecutivi
    max_repeat = 1
    current_repeat = 1
    for i in range(1, length):
        if seq[i] == seq[i-1]:
            current_repeat += 1
            if current_repeat > max_repeat:
                max_repeat = current_repeat
        else:
            current_repeat = 1
    
    # Entropia della sequenza
    entropy = -sum((c/length) * math.log2(c/length) for c in counts.values())
    
    # Feature dei ponti disolfuro: C separati da 2-5 residui
    disulfide_matches = re.findall(r"C.{2,5}C", seq)
    num_disulfide_motifs = len(disulfide_matches)
    
    # Costruzione dizionario finale
    features = {
        "length": length,
        "num_acidi": num_acidi,
        "num_basici": num_basici,
        "num_polari": num_polari,
        "num_idrofobi": num_idrofobi,
        "max_repeat": max_repeat,
        "entropy": entropy,
        "num_disulfide_motifs": num_disulfide_motifs
    }
    
    # Aggiungo le percentuali di ogni residuo
    features.update(aa_percent)
    #print(features)
    return features


In [3]:
def init_matrix (window): #function that initialise the PSPM matrix, with pseudocounts. 
    aminoacids = ["A","R","N","D","C","Q","E","G","H","I",
              "L","K","M","F","P","S","T","W","Y","V"]
    dict_mat = {}
    for i in aminoacids:
        dict_mat[i]= [1 for i in range(window)]
    return dict_mat

In [4]:
def compute_pswm (matrix,list_seq): #function that used the PSPM matrix and a list of sequences and computes the PSWM. Built as a dictionary of lists
    diz_swp = {'A':0.08, 'R':0.06, 'N':0.04, 'D':0.06, 'C':0.01, 'Q':0.04, 'E':0.07, 'G':0.07, 'H':0.02, 'I':0.06, 'L':0.10, 'K':0.06, 'M':0.02, 'F':0.04, 'P':0.05, 'S':0.07, 'T':0.05, 'W':0.01, 'Y':0.03, 'V':0.07} #frequency of residues from SwissProt
    for seq in list_seq:
        for index, res in enumerate(seq[:len(next(iter(matrix.values())))]):  # “For each sequence, go through its residues up to the PSWM length (e.g., 15),and for each residue, increment its count in the corresponding PSWM column.
            if res not in matrix:
                continue
            matrix[res][index] += 1

    
    div = int(len(list_seq))+20

    
    for key in matrix:
        matrix[key] = [np.log2(x/(div*diz_swp[key])) for x in matrix[key]]

    return matrix

In [5]:
def score_window(matrix, seq_window):
    """
    It's the function that calculates the position-specific score of an amino acid window with respect to the PSWM (Position-Specific Weight Matrix) —
    it's then used to slide across the entire sequence and find the point with the highest score, which most likely corresponds to the signal peptide region.
    """
    score = 0 # Initialize the total score to zero,this will accumulate the sum
    for pos in range(len(seq_window)):
        letter = seq_window[pos]
        if letter in matrix:
            # Add the corresponding weight from the PSWM for this amino acid
            # at the specific position to the running total score
            score += matrix[letter][pos]
    return score

In [6]:
def get_highest_pswm_scores(pswm, seqs, window_size=15):
    """
    Given a PSWM (Position-Specific Weight Matrix) and a list of sequences,
    compute the highest log-odds score for each sequence using a sliding window.
    Returns a list containing the highest PSWM score for each sequence.
    """

    scores = []

    for seq in seqs:
        # Skip sequences shorter than the window size
        if len(seq) < window_size:
            continue

        best_score = float('-inf')  # Start with very low value

        # Slide the window
        for i in range(len(seq) - window_size + 1):
            window = seq[i:i + window_size]
            score = 0.0

            for pos, aa in enumerate(window):
                if aa in pswm and pos < len(pswm[aa]):
                    score += pswm[aa][pos]

            # Track the best-scoring window
            if score > best_score:
                best_score = score

        scores.append(best_score)

    return scores


In [7]:
def proline_matrix_numpy(seqs, nterm_len=30):
    """
    Calcola, per ogni sequenza aminoacidica:
      - Il numero di residui 'P' (Proline) nei primi 30 amminoacidi
      - La densità di 'P' (cioè quante 'P' su quanti residui)
    Ritorna una matrice NumPy:
      righe   = sequenze
      colonne = [P_count_30, P_density_30]
    """
    
    # Lista dove salveremo i risultati
    results = []

    # Ciclo su ogni sequenza
    for seq in seqs:
        
        # 2️⃣ Prendiamo solo i primi 30 amminoacidi
        first_30 = seq[:nterm_len]

        # 3️⃣ Calcoliamo quanti 'P' (Proline) ci sono
        p_count = first_30.count('P')

        # 4️⃣ Calcoliamo la lunghezza effettiva (può essere < 30)
        length = len(first_30)

        # 5️⃣ Calcoliamo la densità di Proline
        if length > 0:
            p_density = p_count / length
        else:
            p_density = 0

        # 6️⃣ Creiamo un piccolo vettore [p_count, p_density]
        feature_vector = [p_count, p_density]

        # 7️⃣ Lo aggiungiamo alla lista dei risultati
        results.append(feature_vector)

    # 8️⃣ Convertiamo la lista in una matrice NumPy
    proline_matrix = np.array(results)

    # 9️⃣ Ritorniamo la matrice finale
    return proline_matrix

In [19]:
dataset = pd.read_csv("training_file_all_seq.tsv",sep="\t")
dataset['positive/negative'] =np.where(dataset['SP cleavage']=='False', 0,1)
seqs=dataset["Sequence"].tolist()

In [20]:
#  PROLINE EXTRACTION

final_output = proline_matrix_numpy(seqs)

In [21]:
#  SCORE EXTRACTION

matrix = init_matrix(15)              #initialise the PSWM with 1s and 15 colunmns
pswm = compute_pswm(matrix, seqs)     # build the PSWM
scores = get_highest_pswm_scores(pswm, seqs)   # get best score per sequence

scores=np.array(scores)
scores=scores.reshape(-1,1)

In [22]:
#  FEATURES EXTRACTION

data = [extract_basic_features(seq) for seq in seqs]
df_features = pd.DataFrame(data)
matrix_features = df_features.to_numpy()

In [23]:
# SP COLUMN EXTRACTION

vettore_colonna = dataset['positive/negative'].to_numpy().reshape(-1, 1)

In [24]:
try_matrix = np.hstack([final_output,scores,matrix_features,vettore_colonna])

final_matrix = np.nan_to_num(try_matrix, nan=0.0)

# Columns Description

| Index | Feature                  |
|-------|--------------------------|
| 0     | P_count_30               |
| 1     | P_density_30             |
| 2     | Score                    |
| 3     | length                   |
| 4     | num_acidi                |
| 5     | num_basici               |
| 6     | num_polari               |
| 7     | num_idrofobi             |
| 8     | max_repeat               |
| 9     | entropy                  |
| 10    | num_disulfide_motifs     |
| 11    | %M                       |
| 12    | %K                       |
| 13    | %A                       |
| 14    | %S                       |
| 15    | %V                       |
| 16    | %T                       |
| 17    | %L                       |
| 18    | %G                       |
| 19    | %P                       |
| 20    | %I                       |
| 21    | %R                       |
| 22    | %D                       |
| 23    | %N                       |
| 24    | %F                       |
| 25    | %E                       |
| 26    | %Y                       |
| 27    | %Q                       |
| 28    | %C                       |
| 29    | %H                       |
| 30    | %W                       |
| 31    | %U                       |
| 32    | %X                       |
| 33    | POSITIVE/NEGATIVE        |


In [26]:
final_dataframe = pd.DataFrame(final_matrix)

In [27]:
final_dataframe

,0,1,2,3,4,5,6,7,8,9,...,24,25,26,27,28,29,30,31,32,33
0,1.0,0.033333,7.504679,452.0,51.0,43.0,134.0,151.0,4.0,4.040580,...,0.042035,0.037611,0.053097,0.024336,0.013274,0.002212,0.006637,0.0,0.0,1.0
1,4.0,0.133333,7.335163,390.0,23.0,40.0,104.0,144.0,3.0,3.969724,...,0.020513,0.035897,0.017949,0.051282,0.012821,0.020513,0.012821,0.0,0.0,1.0
2,2.0,0.066667,7.363840,2586.0,299.0,340.0,556.0,1066.0,3.0,4.104638,...,0.036350,0.086234,0.031323,0.025909,0.013148,0.023975,0.006574,0.0,0.0,1.0
3,4.0,0.133333,6.888690,825.0,97.0,99.0,207.0,289.0,5.0,4.189001,...,0.044848,0.052121,0.032727,0.035152,0.020606,0.025455,0.019394,0.0,0.0,1.0
4,0.0,0.000000,9.597996,143.0,9.0,11.0,30.0,57.0,2.0,4.039273,...,0.041958,0.041958,0.027972,0.048951,0.076923,0.013986,0.006993,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8015,0.0,0.000000,6.117120,465.0,74.0,66.0,93.0,171.0,2.0,4.118710,...,0.038710,0.101075,0.021505,0.034409,0.025806,0.019355,0.006452,0.0,0.0,0.0
8016,1.0,0.033333,8.406212,142.0,34.0,15.0,33.0,34.0,4.0,4.067237,...,0.035211,0.169014,0.035211,0.049296,0.049296,0.021127,0.007042,0.0,0.0,0.0
8017,2.0,0.066667,7.176811,445.0,49.0,79.0,106.0,154.0,3.0,4.152206,...,0.065169,0.053933,0.040449,0.026966,0.011236,0.026966,0.024719,0.0,0.0,0.0
8018,1.0,0.033333,7.876031,335.0,60.0,70.0,72.0,115.0,3.0,3.890395,...,0.026866,0.119403,0.000000,0.065672,0.020896,0.011940,0.005970,0.0,0.0,0.0
